## Процесс сбора датасета при помощи OpenAI API  
  
  **Основные функции**  


  | Название | Input | Input format | Output | Output format |
|----------|----------|----------|----------|----------|
| csv2lists| file_path| str | texts, first, length| list, list, list|
| api_request| prompt, AUTHOR, BOOK, TEXT, LENGTH| str, str, str, str, int| API answer| str|
| parsing| api_output| str| [text, PERS, LOC, ACT][t | list of str|
| save2csv | name_of_csv, list_to_save | str, list of lists | - | - |



**Официальная библиотека OpenAI**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


**Необходимые библиотеки**

In [ ]:
import csv
import re
import tqdm
from datetime import datetime
import openai
from openai import OpenAI

## 1. Сбор чанков текста из CSV

In [ ]:
def csv2lists(file_path): # на выходе три списка
  texts = []
  first = []
  length = []

  with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    # чтобы пропустить названия столбцов
    next(reader)
    past_was_deleted = '0'
    for row in reader:
      if int(row[2]) > 156 and int(row[2]) <= 331:
        texts.append(row[0])
        if past_was_deleted == '-1':
          # правило начала главы пересиливает правило пропуска предыдущего
          if str(row[1]) == '1':
            first.append('1')
          else:
            first.append(past_was_deleted)
            past_was_deleted = '0'
        else:
          first.append(str(row[1]))
        length.append(row[2])
      else:
        past_was_deleted == '-1'
    return texts, first, length

# Оформление запроса к API

**Апи ключ получен при создании проекта на [официальном сайте]()**

In [ ]:
client = OpenAI(
    api_key="МОЙ-АПИ-КЛЮЧ",
)

In [ ]:
all_prompts = []

**Финальная версия промпта после множества экспериментов**

In [ ]:
def api_request(BOOK, AUTHOR, TEXT, LENGTH):
  BOOK = BOOK
  AUTHOR = AUTHOR
  #previous_PERS = previous_PERS
  #previous_LOC = previous_LOC
  TEXT = TEXT
  LENGTH = LENGTH
  prompt = "Now you will receive the following information: the title and author \
  of the book and a passage from it.\nYou should output:\n1. The plot of what is \
  happening in the passage\n2. PERS: list the characters involved in the passage\
  \n3. LOC: name the location where the action takes place in the passage\nIf you\
   can’t determine PERS or LOC, output “undefined” for them.\n4. ACT: determine \
   the type of action (rhetorical model) in the passage from the following list: \
   1 action (action occurs), 2 dialogue (characters talk), 3 thoughts (feelings \
   and thoughts description), 4 scene (exposition of a location), 5 description \
   (description of something other than a place), 6 summary (overview of what \
   happened). Output just numbers.\n\nThe book title and the author: {}, {}\nThe\
    passage: {}\n\nIMPORTANT: your answer SHOULD NOT EXCEED in length {} \
    words.".format(BOOK, AUTHOR, TEXT, LENGTH)
  all_damn_prompts.append(prompt)
  answer = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-4o",
  )
  return(answer.choices[0].message.content)

## Распарсить ответ

In [ ]:
def parsing(api_output):
  api_output = api_output.replace("\n", "")
  if "LOC" not in api_output:
    print("Новый идизайн аутпута")
    print("Текст, что не распарсился:\n", api_output)
  else:
    pattern_text = r'1(.*)PERS'
    pattern_PERS = r'PERS(.*?)LOC'
    pattern_LOC = r'LOC(.*?)ACT'
    pattern_ACT = r'ACT(.*)'

  match_text = re.search(pattern_text, api_output)
  match__PERS = re.search(pattern_PERS, api_output)
  match_LOC = re.search(pattern_LOC, api_output)
  match_ACT = re.search(pattern_ACT, api_output)

  #print(match_text)
  #print(match__PERS)
  #print(match_LOC)
  #print(match_ACT)

  try:
    text = match_text.group(1).strip("*: \n.1234")
    PERS = match__PERS.group(1).strip("*: \n.1234")
    LOC = match_LOC.group(1).strip("*: \n.1234")
    ACT = match_ACT.group(1).strip("*: \n.")
  except Exception as e:
    print(e)
    print("Текст, что не распарсился:\n", api_output)

  return [text, PERS, LOC, ACT]


In [ ]:
parsing('1. The plot of what is happening in the passage\n2. PERS: list the characters involved in the passage\n3. LOC: name the location where the action takes place in the passage\n4. ACT: determine the type of action (rhetorical model) in the passage from the following list: 1 action (action occurs), 2 dialogue (characters talk), 3 thoughts (feelings and thoughts description), 4 scene (exposition of a location), 5 description (description of something other than a place), 6 summary (overview of what happened)')

1. The plot of what is happening in the passage2. PERS: list the characters involved in the passage3. LOC: name the location where the action takes place in the passage4. ACT: determine the type of action (rhetorical model) in the passage from the following list: 1 action (action occurs), 2 dialogue (characters talk), 3 thoughts (feelings and thoughts description), 4 scene (exposition of a location), 5 description (description of something other than a place), 6 summary (overview of what happened)
<re.Match object; span=(0, 54), match='1. The plot of what is happening in the passage2.>
<re.Match object; span=(50, 105), match='PERS: list the characters involved in the passage>
<re.Match object; span=(102, 174), match='LOC: name the location where the action takes pla>
<re.Match object; span=(171, 502), match='ACT: determine the type of action (rhetorical mod>


['The plot of what is happening in the passage',
 'list the characters involved in the passage',
 'name the location where the action takes place in the passage',
 'determine the type of action (rhetorical model) in the passage from the following list: 1 action (action occurs), 2 dialogue (characters talk), 3 thoughts (feelings and thoughts description), 4 scene (exposition of a location), 5 description (description of something other than a place), 6 summary (overview of what happened)']

## Сбор в один код запроса

In [ ]:
file_path = str(1016) + '.csv'

In [ ]:
final_csv = [['chunk1','chunk2','author','book','PERS','LOC','ACT']]

start_time = datetime.now()
texts, first, length = csv2lists(file_path)
texts = new_texts
AUTHOR = 'J R R Tolkien'
BOOK = 'Hobbit or There and Back Again'
LENGTH = 165
for i in  tqdm.tqdm(range(len(texts))):
  TEXT = texts[i]
  start = first[i]
  csv_line = parsing(api_request(BOOK, AUTHOR, TEXT, LENGTH))
  final_line = [TEXT, csv_line[0], AUTHOR, BOOK, csv_line[1], csv_line[2], csv_line[3]]
  final_csv.append(final_line)
stop_time = datetime.now()
elapsed_time = stop_time - start_time
print(f"The code worked for: {elapsed_time}")

100%|██████████| 206/206 [08:50<00:00,  2.58s/it]

The code worked for: 0:08:50.762252


## Пример выдачи

In [ ]:
final_csv

[['chunk1', 'chunk2', 'author', 'book', 'PERS', 'LOC', 'ACT'],
 ['In a hole in the ground there lived a hobbit. Not a nasty, dirty, wet hole, filled with the ends of worms and an oozy smell, nor yet a dry, bare, sandy hole with nothing in it to sit down on or to eat: it was a hobbit-hole, and that means comfort.\nIt had a perfectly round door like a porthole, painted green, with a shiny yellow brass knob in the exact middle. The door opened on to a tube-shaped hall like a tunnel: a very comfortable tunnel without smoke, with panelled walls, and floors tiled and carpeted, provided with polished chairs, and lots and lots of pegs for hats and coats - the hobbit was fond of visitors. The tunnel wound on and on, going fairly but not quite straight into the side of the hill - The Hill, as all the people for many miles round called it - and many little round doors opened out of it, first on one side and then on another. No going upstairs for the hobbit: bedrooms, bathrooms, cellars, pantries 

## Сохранение в файл

In [ ]:
def save2csv(name_of_csv, list_to_save):
  with open(name_of_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(list_to_save)

In [ ]:
save2csv("1016_Hobbit.csv", final_csv)

In [ ]:
# Дополнительное сохранение промптов
with open("prompts_Sense_Hobbit.txt", "w") as file:
    for item in all_prompts:
        file.write(f"{item}\n\n")